In [1]:
import pandas as pd
data = pd.read_csv("every_worker.csv",encoding='utf-8-sig')

In [2]:
data = data[['pid','year','p_age','p_econstat','p_edu','p_job_status','p_job_begin','p_employ_type','p_hours','job_name','산업명','p_jobtype','p_sex','p_wage']]

In [3]:
data = data.dropna()#40444개 ->39774개

In [4]:
def age_group(age):
    if age <= 29:
        return 1
    elif age <= 39:
        return 2
    elif age <= 49:
        return 3
    elif age <= 59:
        return 4
    else:
        return 5

In [5]:
def map_edu(val):
    if val in [1, 2]:
        return 1
    elif val == 3:
        return 2
    elif val in [4, 5]:
        return 3
    elif val == 6:
        return 4
    else:
        return pd.NA  # 결측치 처리

In [6]:
data["hourly_wage"] = (data["p_wage"]*10000) / (data["p_hours"] * 4.345)
data['year'] = pd.to_numeric(data['year'], errors='coerce')
data['p_job_begin'] = data['p_job_begin'].str.replace(r'm.*', '', regex=True)
data['p_job_begin'] = pd.to_numeric(data['p_job_begin'], errors='coerce')
data['경력기간'] = data['year'] - data['p_job_begin']
data['경력기간'] = data.groupby('pid')['경력기간'].transform('max')
data['different_year'] = data['year'].map({2017: 0, 2018: 1, 2019: 0, 2020: 1})
data['p_sex'] = data['p_sex'].map({1: 0, 2: 1})
data['p_age'] = data['p_age'].apply(age_group)
data['p_edu'] = data['p_edu'].apply(map_edu)
data['p_job_status'] = data['p_job_status'].replace(2, 0)
data = data[~data['p_job_status'].isin([3, 4, 5])].copy()

## 업종, 직종 전처리

In [7]:
# 2. 직종 분류 함수
def classify_job(job_name):
    if pd.isna(job_name):
        return "기타"
    elif any(x in job_name for x in ["관리", "전문", "연구", "의사", "변호사", "교수", "엔지니어", "기획"]):
        return "관리자, 전문가 및 관련종사자"
    elif any(x in job_name for x in ["사무", "행정", "회계", "총무", "비서"]):
        return "사무종사자"
    elif any(x in job_name for x in ["판매", "서비스", "요리", "식당", "접객", "농업", "어업", "호텔", "숙박"]):
        return "서비스, 판매종사자 등"
    elif any(x in job_name for x in ["기계", "조작", "조립", "정비", "운전", "생산", "설비", "제조", "전기", "공", "배선"]):
        return "기능, 기계조작, 조립종사자"
    elif any(x in job_name for x in ["청소", "미화", "노무", "단순", "경비", "하역"]):
        return "단순노무종사자"
    else:
        return "기타"

# 3. 업종 분류 함수
def classify_industry(ind_name):
    if pd.isna(ind_name):
        return "기타"
    elif any(x in ind_name for x in ["농업", "광업", "제조", "건설", "전기", "수도", "에너지"]):
        return "농업, 광업, 제조업, 전기, 수도, 건설"
    elif any(x in ind_name for x in ["도매", "소매", "유통", "운수", "물류", "택배", "숙박", "음식", "식당"]):
        return "도매, 운수, 숙박"
    elif any(x in ind_name for x in ["정보", "통신", "IT", "금융", "보험", "부동산", "관리", "시설"]):
        return "정보, 금융, 부동산, 시설관리"
    elif any(x in ind_name for x in ["교육", "병원", "보건", "의료", "공공", "연구", "대학", "행정"]):
        return "전문, 공공, 교육, 보건"
    elif any(x in ind_name for x in ["예술", "문화", "종교", "협회", "국제", "자가", "외교"]):
        return "예술, 협회, 자가, 국제"
    elif any(x in ind_name for x in ["청소", "방제", "환경", "위생"]):
        return "청소 및 환경관리 서비스"
    else:
        return "기타"

In [8]:
# 4. 분류 적용
data['직종'] = data['job_name'].apply(classify_job)
data['업종'] = data['산업명'].apply(classify_industry)

## 최저임금영향집단, 최저임금미만집단 분류

In [9]:
# 연도별 최저임금
min_wage_dict = {
    2017: 6470,
    2018: 7530,
    2019: 8350,
    2020: 8590
}

# 시급 대비 최저임금 비율 계산
data['minwage_ratio'] = data.apply(
    lambda row: row['hourly_wage'] / min_wage_dict.get(row['year'], 1)
    if pd.notna(row['hourly_wage']) else pd.NA,
    axis=1
)

# 개인별 연도별 시급 비율 pivot 테이블 생성
pivot = data.pivot_table(index='pid', columns='year', values='minwage_ratio')

# 함수 정의: 영향집단 vs 미만집단
def label_minwage_group(row):
    # 최저임금 영향집단
    cond_before = (row.get(2017, 0) >= 1.0) or (row.get(2019, 0) >= 1.0)
    cond_after  = (row.get(2018, 0) < 1.0) or (row.get(2020, 0) < 1.0)
    if cond_before and cond_after:
        return "최저임금 영향집단"

    # 최저임금 미만집단: 2017년 기준 80% 이상이며, 전체 연도 모두 0.8 이상
    min_all_above_80 = all((pd.isna(v) or v >= 0.8) for v in row)
    if row.get(2017, 0) >= 0.8 and min_all_above_80:
        return "최저임금 미만집단"

    # 나머지는 분석 제외
    return pd.NA

# 집단 부여
pivot['임금집단'] = pivot.apply(label_minwage_group, axis=1)

# 원본 데이터와 병합
data = data.merge(pivot[['임금집단']], left_on='pid', right_index=True, how='left')

In [10]:
data1 = data[data['임금집단']=='최저임금 영향집단']
data2 = data[data['임금집단']=='최저임금 미만집단']
print(len(data1),len(data2))

2815 18360


In [19]:
import pandas as pd
import numpy as np

# 1. 정규직/비정규직 분류 (p_job_status: 0=정규직, 1=비정규직)
data2['고용형태'] = data2['p_job_status'].map({0.0: '비정규직', 1.0: '정규직'})

# 분석 변수 설정
group_vars = {
    '연령': 'p_age',  # 이미 그룹화된 값 (1~5)
    '학력': 'p_edu',  # map_edu() 적용된 값 (1~4)
    '근로시간': lambda df: np.where(df['p_hours'] <= 40, '주40시간이하', '주40시간초과'),
    '경력년수': lambda df: pd.cut(df['경력기간'], bins=[-1, 4, 9, 14, np.inf],
                               labels=['5년 미만', '5~10년미만', '10~15년미만', '15년이상']),
    '업종': '업종',
    '직종': '직종'
}

# 기초통계 요약 결과를 저장할 리스트
summary_table = []

# 연도별 & 고용형태별 반복
for year in [2017, 2019]:
    for emp_type in ['정규직', '비정규직']:
        subset = data2[(data2['year'] == year) & (data2['고용형태'] == emp_type)]

        row = {
            '연도': year,
            '고용형태': emp_type,
            '개체 수': len(subset)
        }

        for label, var in group_vars.items():
            col_data = pd.Series(var(subset)) if callable(var) else subset[var]
            value_counts = col_data.value_counts(normalize=True).mul(100).round(1)
            for category, percent in value_counts.items():
                row[f'{label}_{category}'] = percent

        summary_table.append(row)

# 결과 데이터프레임 생성
summary_df = pd.DataFrame(summary_table)

# 결과 확인
print(summary_df)

     연도  고용형태  개체 수  연령_3  연령_2  연령_4  연령_1  연령_5  학력_4  학력_3  ...  \
0  2017   정규직  4667  32.6  31.4  17.5  13.2   5.3  40.1  27.6  ...   
1  2017  비정규직   773  24.7  18.4  19.3  19.1  18.5  27.4  22.1  ...   
2  2019   정규직  3839  36.7  28.0  21.3   7.6   6.4  40.9  27.5  ...   
3  2019  비정규직   482  22.2  20.5  21.2  11.4  24.7  28.8  19.5  ...   

   업종_도매, 운수, 숙박  업종_정보, 금융, 부동산, 시설관리  업종_예술, 협회, 자가, 국제  업종_청소 및 환경관리 서비스  \
0           13.9                   9.8                0.7               0.3   
1           21.1                  12.0                0.5               1.0   
2           14.0                   9.9                0.8               0.4   
3           19.7                  13.1                0.6               0.6   

   직종_사무종사자  직종_기타  직종_기능, 기계조작, 조립종사자  직종_관리자, 전문가 및 관련종사자  직종_서비스, 판매종사자 등  \
0      28.4   26.4                25.8                  9.2              8.1   
1      10.5   31.8                13.3                  8.9             26.4   
2      28.1  

<ipython-input-19-82d15fdff2aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['고용형태'] = data2['p_job_status'].map({0.0: '비정규직', 1.0: '정규직'})


In [23]:
data1['p_age'].value_counts()

,count
p_age,
5,976
4,659
3,554
2,357
1,269


In [ ]:
data